In [2]:
import random

class State:
    def __init__(self, pieces=None, enemy_pieces=None):
        self.pieces = pieces if pieces != None else [0] * 9
        self.enemy_pieces = enemy_pieces if enemy_pieces != None else [0] * 9
    
    def pieces_count(self, pieces):
        count = 0
        for i in pieces:
            if i == 1:
                count += 1
        return count
    
    def is_lose(self):
        def is_comp(x, y, dx, dy):
            for k in range(3):
                if y < 0 or 2 < y or x < 0 or 2 < x or self.enemy_pieces[x+y*3] == 0:
                    return False
                x += dx
                y += dy
            return True
        if is_comp(0, 0, 1, 1) or is_comp(0, 2, 1, -1): #ななめ方向の判定
            return True
        for i in range(3): #たてよこ方向の判定
            if is_comp(0, i, 1, 0) or is_comp(i, 0, 0, 1):
                return True
        return False
    def is_draw(self):
        return self.pieces_count(self.pieces) + self.pieces_count(self.enemy_pieces) == 9
    
    def is_done(self):
        return self.is_lose() or self.is_draw()
    
    def next(self,action):
        pieces = self.pieces.copy()
        pieces[action] = 1
        return State(self.enemy_pieces, pieces)
    
    def legal_actions(self):
        actions = []
        for i in range(9):
            if self.pieces[i] == 0 and self.enemy_pieces[i] == 0:
                actions.append(i)
        return actions
    
    def is_first_player(self):
        return self.pieces_count(self.pieces) == self.pieces_count(self.enemy_pieces)
    
    def __str__(self):
        ox = ('o','x') if self.is_first_player() else ('x', 'o')
        str = ''
        
        for i in range(9):
            if self.pieces[i] == 1:
                str += ox[0]
            elif self.enemy_pieces[i] == 1:
                str += ox[1]
            else:
                str += '-'
            if i % 3 == 2:
                str += '\n'
        return str

In [4]:
def random_action(state):
    legal_actions = state.legal_actions()
    return legal_actions[random.randint(0, len(legal_actions)-1)]

In [7]:
state = State()

while True:
    if state.is_done():
        break
    action = random_action(state)
    state = state.next(action)
    print(state)
    print()

---
--o
---


-x-
--o
---


-x-
--o
-o-


-x-
-xo
-o-


ox-
-xo
-o-


ox-
xxo
-o-


ox-
xxo
-oo


ox-
xxo
xoo


oxo
xxo
xoo




In [14]:
def mini_max(state):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    best_score = -float('inf')
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_score = score
    return best_score

def mini_max_action(state):
    best_action = 0
    best_score = -float('inf')
    
    str = ['','']
    for action in state.legal_actions():
        score = -mini_max(state.next(action))
        if score > best_score:
            best_action = action
            best_score = score
        str[0] = '{}{:2d},'.format(str[0],action)
        str[1] = '{}{:2d},'.format(str[1],score)
    print('action:',str[0],'\nscore:',str[1])
    return best_action

In [15]:
state = State()

while True:
    if state.is_done():
        break
    if state.is_first_player():
        action = mini_max_action(state)
    else:
        action = random_action(state)
    state = state.next(action)
    print(state)
    print()

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, 
score:  0, 0, 0, 0, 0, 0, 0, 0, 0,
o--
---
---


o-x
---
---


action:  1, 3, 4, 5, 6, 7, 8, 
score: -1, 1, 0, 0, 1, 0, 1,
o-x
o--
---


o-x
ox-
---


action:  1, 5, 6, 7, 8, 
score: -1,-1, 1,-1,-1,
o-x
ox-
o--




In [16]:
def mini_max_plus(state, limit):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    best_score = -float('inf')
    
    for action in state.legal_actions():
        score = -mini_max_plus(state.next(action), -best_score)
        if score > best_score:
            best_score = score
        if best_score >= limit:
            return best_score
    return best_score

In [22]:
def alpha_beta(state, alpha, beta):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action), -beta, -alpha)
        if score > alpha:
            alpha = score
        if alpha > beta:
            return alpha
    return alpha

In [24]:
def alpha_beta_action(state):
    beta_action = 0
    alpha = -float('inf')
    str = ['','']
    for action in state.legal_actions():
        score = -alpha_beta(state.next(action),-float('inf'),-alpha)
        if score > alpha:
            best_action = action
            alpha = score
        str[0] = '{}{:2d},'.format(str[0], action)
        str[1] = '{}{:2d},'.format(str[1], score)
    print('action:',str[0],'score:',str[1],'\n')
    return best_action

In [25]:
state = State()

while True:
    if state.is_done():
        break
    if state.is_first_player():
        action = alpha_beta_action(state)
    else:
        action = mini_max_action(state)
    state = state.next(action)
    print(state)
    print()

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

o--
---
---


action:  1, 2, 3, 4, 5, 6, 7, 8, 
score: -1,-1,-1, 0,-1,-1,-1,-1,
o--
-x-
---


action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

oo-
-x-
---


action:  2, 3, 5, 6, 7, 8, 
score:  0,-1,-1,-1,-1,-1,
oox
-x-
---


action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

oox
-x-
o--


action:  3, 5, 7, 8, 
score:  0,-1,-1,-1,
oox
xx-
o--


action:  5, 7, 8, score:  0,-1,-1, 

oox
xxo
o--


action:  7, 8, 
score:  0, 0,
oox
xxo
ox-


action:  8, score:  0, 

oox
xxo
oxo




In [32]:
def playout(state):
    if state.is_lose():
        return -1
    if state.is_draw():
        return 0
    return -playout(state.next(random_action(state)))

In [38]:
def argmax(collections, key=None):
    return collections.index(max(collections))

def mcs_action(state):
    legal_actions = state.legal_actions()
    values = [0] * len(legal_actions)
    for i, action in enumerate(legal_actions):
        for _ in range(10):
            values[i] += -playout(state.next(action))
    return legal_actions[argmax(values)]

In [42]:
EP_GAME_COUNT = 100

def first_player_point(ended_state):
    if ended_state.is_lose():
        return 0 if ended_state.is_first_player() else 1
    return 0.5

def play(next_actions):
    state = State()
    while True:
        if state.is_done():
            break
        next_action = next_actions[0] if state.is_first_player() else next_actions[1]
        action = next_action(state)
        state = state.next(action)
    return first_player_point(state)

def evaluate_alg(label, next_actions):
    total_point = 0
    for i in range(EP_GAME_COUNT):
        if i % 2 == 0:
            total_point += play(next_actions)
        else:
            total_point += 1 - play(list(reversed(next_actions)))
        print('Eval {}/{}'.format(i + 1, EP_GAME_COUNT),end=' ')
        print('')
    average = total_point / EP_GAME_COUNT
    print(label.format(average))
next_actions = (mcs_action, random_action)
evaluate_alg('VS_RANDOM {:.3f}',next_actions)

Eval 1/100 
Eval 2/100 
Eval 3/100 
Eval 4/100 
Eval 5/100 
Eval 6/100 
Eval 7/100 
Eval 8/100 
Eval 9/100 
Eval 10/100 
Eval 11/100 
Eval 12/100 
Eval 13/100 
Eval 14/100 
Eval 15/100 
Eval 16/100 
Eval 17/100 
Eval 18/100 
Eval 19/100 
Eval 20/100 
Eval 21/100 
Eval 22/100 
Eval 23/100 
Eval 24/100 
Eval 25/100 
Eval 26/100 
Eval 27/100 
Eval 28/100 
Eval 29/100 
Eval 30/100 
Eval 31/100 
Eval 32/100 
Eval 33/100 
Eval 34/100 
Eval 35/100 
Eval 36/100 
Eval 37/100 
Eval 38/100 
Eval 39/100 
Eval 40/100 
Eval 41/100 
Eval 42/100 
Eval 43/100 
Eval 44/100 
Eval 45/100 
Eval 46/100 
Eval 47/100 
Eval 48/100 
Eval 49/100 
Eval 50/100 
Eval 51/100 
Eval 52/100 
Eval 53/100 
Eval 54/100 
Eval 55/100 
Eval 56/100 
Eval 57/100 
Eval 58/100 
Eval 59/100 
Eval 60/100 
Eval 61/100 
Eval 62/100 
Eval 63/100 
Eval 64/100 
Eval 65/100 
Eval 66/100 
Eval 67/100 
Eval 68/100 
Eval 69/100 
Eval 70/100 
Eval 71/100 
Eval 72/100 
Eval 73/100 
Eval 74/100 
Eval 75/100 
Eval 76/100 
Eval 77/100 
Eval 78/

In [43]:
next_actions = (mcs_action, alpha_beta_action)
evaluate_alg('VS_RANDOM {:.3f}',next_actions)

action:  1, 2, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 7, 8, score: -1,-1, 0,-1,-1,-1, 

action:  1, 2, 7, 8, score:  0, 0, 0, 0, 

action:  2, 8, score: -1, 0, 

Eval 1/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

action:  3, 7, 8, score:  1,-1,-1, 

Eval 2/100 
action:  0, 1, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  0, 3, 5, 6, 7, 8, score:  0,-1,-1,-1,-1,-1, 

action:  3, 6, 7, 8, score: -1,-1,-1, 1, 

Eval 3/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 5, 7, 8, score:  1, 1, 1, 1, 1, 1, 1, 

action:  3, 4, 5, 7, 8, score: -1, 1,-1,-1,-1, 

action:  3, 5, 7, score: -1,-1, 1, 

Eval 4/100 
action:  0, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  2, 4, 5, 6, 7, 8, score: -1, 0, 0, 0, 0, 0, 

action:  2, 5, 6, 8, score:  

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 4, 5, 6, 7, 8, score:  1, 1, 1, 1, 1, 1, 1, 

action:  2, 4, 6, 7, 8, score:  1, 1,-1,-1,-1, 

Eval 38/100 
action:  0, 1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  2, 3, 5, 6, 7, 8, score: -1,-1,-1,-1, 0,-1, 

action:  2, 5, 6, 8, score: -1, 0,-1,-1, 

action:  2, 8, score:  0,-1, 

Eval 39/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 5, 6, 7, score: -1, 1, 1, 1, 1, 1, 1, 

action:  3, 4, 5, 6, 7, score:  0, 0, 0, 1, 1, 

action:  3, 4, 5, score:  1, 1,-1, 

Eval 40/100 
action:  0, 1, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  0, 3, 5, 6, 7, 8, score:  0,-1,-1,-1,-1,-1, 

action:  3, 5, 6, 7, score: -1, 0,-1,-1, 

action:  6, 7, score:  0, 0, 

Eval 41/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 3, 4, 5, 6, 7, 8, score: -1, 1, 1, 1, 1, 1, 1, 

action:  1, 4, 5, 7

action:  0, 1, 2, 3, 4, 5, 6, 8, score: -1, 0, 0, 0, 0, 0, 0, 0, 

action:  0, 2, 3, 4, 5, 8, score: -1,-1,-1,-1,-1, 0, 

action:  0, 3, 4, 5, score: -1,-1, 0,-1, 

action:  3, 5, score:  0,-1, 

Eval 77/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 3, 4, 5, 6, 7, 8, score: -1, 1, 1, 1, 1, 1, 1, 

action:  1, 4, 5, 7, 8, score: -1, 1,-1,-1,-1, 

action:  1, 5, 7, score: -1, 1,-1, 

Eval 78/100 
action:  0, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  3, 4, 5, 6, 7, 8, score:  1, 1, 1, 1, 1, 1, 

action:  5, 6, 7, 8, score: -1, 1,-1,-1, 

Eval 79/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 5, 7, 8, score:  1, 1, 1, 1, 1, 1, 1, 

action:  2, 3, 4, 5, 7, score:  1,-1,-1,-1, 1, 

Eval 80/100 
action:  0, 1, 2, 3, 4, 6, 7, 8, score: -1,-1, 0, 0, 0, 0, 0, 0, 

action:  0, 1, 3, 6, 7, 8, score: -1,-1, 0,-1,-1,-1, 

action:  0, 1, 6, 8, score: -1, 0,-1,-1, 

action:  6, 8, scor

In [66]:
import math

def mcts_action(state):
    class Node:
        def __init__(self, state):
            self.state = state
            self.w = 0
            self.n = 0
            self.child_nodes = None
        
        def evaluate(self):
            if self.state.is_done():
                value = -1 if self.state.is_lose() else 0
            
                self.w += value
                self.n += 1
                return value
        
            if not self.child_nodes:
                value = playout(self.state)
                self.w += value
                self.n += 1
                if self.n == 10:
                    self.expand()
                return value
            else:
                value = -self.next_child_node().evaluate()
                self.w += value
                self.n += 1
                return value
        
        def expand(self):
            legal_actions = self.state.legal_actions()
            self.child_nodes = []
            for action in legal_actions:
                self.child_nodes.append(Node(self.state.next(action)))
        
        def next_child_node(self):
            for child_node in self.child_nodes:
                if child_node.n == 0:
                    return child_node
                
            t = 0
            for c in self.child_nodes:
                t += c.n
            ucb1_values = []
            for child_node in self.child_nodes:
                
                ucb1_values.append(-child_node.w/child_node.n + (2 * math.log(t)/child_node.n)**0.5)
            return self.child_nodes[argmax(ucb1_values)]
        
            
    root_node = Node(state)
    root_node.expand()
    
    for _ in range(100):
        root_node.evaluate()
    
    legal_actions = state.legal_actions()
    n_list = []
    
    for c in root_node.child_nodes:
        n_list.append(c.n)
    return legal_actions[argmax(n_list)]
    
    

In [68]:
next_actions = (mcts_action, alpha_beta_action)
evaluate_alg('VS_RANDOM {:.3f}',next_actions)

action:  0, 1, 2, 3, 4, 5, 7, 8, score: -1,-1,-1,-1, 0, 0, 0, 0, 

action:  0, 1, 2, 3, 5, 7, score: -1,-1,-1,-1,-1, 0, 

action:  0, 2, 3, 5, score:  0, 0, 0, 0, 

action:  2, 3, score:  0,-1, 

Eval 1/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

action:  5, 7, 8, score:  0,-1,-1, 

action:  8, score:  0, 

Eval 2/100 
action:  0, 1, 2, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  2, 4, 5, 6, 7, 8, score: -1, 0, 0, 0, 0, 0, 

action:  2, 5, 6, 7, score:  0, 0, 0, 0, 

action:  5, 7, score: -1, 0, 

Eval 3/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 3, 4, 5, 6, 7, 8, score: -1, 1, 1, 1, 1, 1, 1, 

action:  1, 4, 5, 7, 8, score: -1, 1,-1,-1,-1, 

action:  1, 5, 7, score: -1, 1,-1, 

Eval 4/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, score: -1,-1,-1,-1, 0, 0, 0, 0, 

action:  0, 1, 2, 3, 5, 6, s

action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

action:  5, 7, 8, score:  0,-1,-1, 

action:  8, score:  0, 

Eval 38/100 
action:  0, 1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 7, 8, score: -1, 0,-1,-1,-1,-1, 

action:  3, 5, 7, 8, score: -1,-1, 0,-1, 

action:  5, 8, score:  0,-1, 

Eval 39/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

action:  5, 7, 8, score:  0,-1,-1, 

action:  8, score:  0, 

Eval 40/100 
action:  1, 2, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 2, 5, 6, 7, 8, score: -1,-1,-1, 0,-1,-1, 

action:  1, 5, 7, 8, score:  0,-1,-1,-1, 

action:  5, 8, score:  0, 0, 

Eval 41/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0

action:  1, 2, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 7, 8, score: -1,-1, 0,-1,-1,-1, 

action:  1, 2, 7, 8, score:  0, 0, 0, 0, 

action:  2, 8, score: -1, 0, 

Eval 75/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 4, 6, 7, 8, score: -1, 1, 1, 1, 1, 1, 1, 

action:  1, 3, 6, 7, 8, score:  1, 1,-1,-1,-1, 

Eval 76/100 
action:  1, 2, 3, 4, 5, 6, 7, 8, score: -1,-1,-1, 0, 0, 0, 0, 0, 

action:  1, 3, 5, 6, 7, 8, score:  0,-1,-1,-1,-1,-1, 

action:  3, 5, 6, 8, score:  0, 0, 0, 0, 

action:  6, 8, score: -1, 0, 

Eval 77/100 
action:  0, 1, 2, 3, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 3, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 

action:  3, 5, 6, 7, 8, score: -1,-1, 0,-1,-1, 

action:  5, 7, 8, score:  0,-1,-1, 

action:  8, score:  0, 

Eval 78/100 
action:  0, 1, 2, 4, 5, 6, 7, 8, score:  0, 0, 0, 0, 0, 0, 0, 0, 

action:  1, 2, 5, 6, 7, 8, score: -1,-1, 0,-1,-1,-1, 

action